In [18]:
base_dataset_path = '../../datasets/ren_et_al/'

dataset_name_path = 'figer/'

types_list_path = 'all_types.txt'

train_path = 'train.json'
dev_path = 'dev.json'
test_path = 'test.json'

# suffix = '.json'
suffix = '_dev.json'
# suffix = '_test.json'

dir_for_partitioned_datasets = '../../datasets/datasets_for_incremental_training/'

In [19]:
# import types
with open(base_dataset_path + dataset_name_path + types_list_path, 'r') as inp:
    types_list = [l.replace('\n', '') for l in inp.readlines()]
types_list

['/art',
 '/art/film',
 '/astral_body',
 '/award',
 '/biology',
 '/body_part',
 '/broadcast',
 '/broadcast/tv_channel',
 '/broadcast_network',
 '/broadcast_program',
 '/building',
 '/building/airport',
 '/building/dam',
 '/building/hospital',
 '/building/hotel',
 '/building/library',
 '/building/power_station',
 '/building/restaurant',
 '/building/sports_facility',
 '/building/theater',
 '/chemistry',
 '/computer',
 '/computer/algorithm',
 '/computer/programming_language',
 '/disease',
 '/education',
 '/education/department',
 '/education/educational_degree',
 '/event',
 '/event/attack',
 '/event/election',
 '/event/military_conflict',
 '/event/natural_disaster',
 '/event/protest',
 '/event/sports_event',
 '/event/terrorist_attack',
 '/finance',
 '/finance/currency',
 '/finance/stock_exchange',
 '/food',
 '/game',
 '/geography',
 '/geography/glacier',
 '/geography/island',
 '/geography/mountain',
 '/god',
 '/government',
 '/government/government',
 '/government/political_party',
 '/gov

In [20]:
# read dataset
import json
with open(base_dataset_path + dataset_name_path + dev_path, 'r') as inp:
    lines = [json.loads(l) for l in inp.readlines()]

In [21]:
len(lines)

1094

In [22]:
# declare types to filter
types_to_filter = ['/location']

In [23]:
from collections import defaultdict
from tqdm import tqdm

# create dataset with the examples annotated with only the types_to_filter 
# or annotated with other types that are not sons of types_to_filter

new_dataset_partitions = defaultdict(list)
new_dataset_partitions['only_father'] = defaultdict(list)
new_dataset_partitions['father_and_sons'] = defaultdict(list)

types_only_partitions = defaultdict(list)
types_only_partitions['only_father'] = defaultdict(list)                
types_only_partitions['father_and_sons'] = defaultdict(list)                

for l in tqdm(lines):
    line_types = l['y_str']
    if all([t not in line_types for t in types_to_filter]):
        new_dataset_partitions['disjoint'].append(l)
        types_only_partitions['disjoint'].append(line_types)
    else:
        for t in types_to_filter:
            if sum([t in line_type for line_type in line_types ]) == 1:
                new_dataset_partitions['only_father'][t].append(l)
                types_only_partitions['only_father'][t].append(line_types)
            else:
                for lt in line_types:
                    if t in lt and lt != t:
                        new_dataset_partitions['father_and_sons'][lt].append(l)
                        types_only_partitions['father_and_sons'][lt].append(line_types)
                

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1094/1094 [00:00<00:00, 544960.64it/s]


In [24]:
types_only_partitions['disjoint']

[['/government',
  '/government/political_party',
  '/organization',
  '/organization/terrorist_organization'],
 ['/music'],
 ['/person', '/person/artist'],
 ['/government', '/government/government'],
 ['/event', '/event/election'],
 ['/person'],
 ['/people', '/people/ethnicity'],
 ['/event'],
 ['/astral_body'],
 ['/person'],
 ['/person', '/person/director'],
 ['/language', '/people', '/people/ethnicity'],
 ['/organization', '/organization/sports_team'],
 ['/organization', '/organization/sports_team'],
 ['/title'],
 ['/person', '/person/author'],
 ['/broadcast_network',
  '/organization',
  '/organization/company',
  '/person',
  '/person/artist'],
 ['/person', '/person/actor', '/person/artist', '/person/musician'],
 ['/organization', '/organization/company'],
 ['/written_work'],
 ['/broadcast_program'],
 ['/person'],
 ['/government_agency', '/organization'],
 ['/government_agency', '/organization'],
 ['/government_agency', '/organization'],
 ['/organization'],
 ['/organization', '/org

In [25]:
types_only_partitions['father_and_sons'].keys()

dict_keys(['/location/city', '/location/country', '/location/body_of_water', '/location/cemetery', '/location/province', '/location/county', '/location/bridge'])

In [26]:
types_only_partitions['father_and_sons']['/location/country']

[['/location', '/location/city', '/location/country'],
 ['/location',
  '/location/country',
  '/people',
  '/people/ethnicity',
  '/person',
  '/person/artist'],
 ['/location', '/location/country', '/person', '/person/artist'],
 ['/government_agency',
  '/language',
  '/location',
  '/location/cemetery',
  '/location/country',
  '/person',
  '/person/artist',
  '/title'],
 ['/location', '/location/country'],
 ['/location', '/location/country'],
 ['/location', '/location/country', '/person', '/person/author'],
 ['/location', '/location/country', '/person', '/person/author'],
 ['/location',
  '/location/country',
  '/people',
  '/people/ethnicity',
  '/person',
  '/person/artist'],
 ['/government', '/government/government', '/location', '/location/country'],
 ['/location',
  '/location/country',
  '/organization',
  '/organization/company',
  '/people',
  '/people/ethnicity'],
 ['/location',
  '/location/body_of_water',
  '/location/city',
  '/location/country'],
 ['/government_agency',

In [27]:
types_only_partitions['only_father']

defaultdict(list,
            {'/location': [['/location',
               '/person',
               '/person/actor',
               '/person/artist',
               '/person/musician'],
              ['/location', '/organization', '/organization/sports_team'],
              ['/location', '/organization', '/organization/sports_team'],
              ['/location'],
              ['/building',
               '/location',
               '/organization',
               '/organization/educational_institution'],
              ['/location'],
              ['/location'],
              ['/location'],
              ['/location'],
              ['/location'],
              ['/location'],
              ['/location'],
              ['/location',
               '/organization',
               '/organization/educational_institution'],
              ['/location',
               '/organization',
               '/organization/educational_institution'],
              ['/location',
               '/organiza

In [28]:
len(lines)

1094

In [29]:
len(types_only_partitions['disjoint'])

633

In [30]:
len(new_dataset_partitions['disjoint'])

633

In [31]:
len(new_dataset_partitions['only_father']['/location'])

149

In [32]:
types_only_partitions['father_and_sons'].keys()

dict_keys(['/location/city', '/location/country', '/location/body_of_water', '/location/cemetery', '/location/province', '/location/county', '/location/bridge'])

In [16]:
for k in types_only_partitions['father_and_sons'].keys():
    print(len(types_only_partitions['disjoint']) + len(types_only_partitions['only_father']['/location']) + len(types_only_partitions['father_and_sons'][k]))

2281492
2165406
2002722
1997513
1962637
1946370
1912979


In [17]:
pretraining_datasets = {}
for father in types_to_filter:
    with open(dir_for_partitioned_datasets + dataset_name_path + 'pretraining_' + father.replace('/', '') + suffix, 'w') as out:
        for disjoint_line in tqdm(new_dataset_partitions['disjoint']):
            json.dump(disjoint_line, out)
            out.write('\n')
        for of_line in tqdm(new_dataset_partitions['only_father'][father]):
            json.dump(of_line, out)
            out.write('\n')
#     pretraining_dataset = {}
    for son in new_dataset_partitions['father_and_sons'].keys():
        with open(dir_for_partitioned_datasets + dataset_name_path + 'incremental_' + son.replace('/', '') + suffix, 'w') as out:
            for son_examples in tqdm(new_dataset_partitions['father_and_sons'][son]):
                son_examples['y_str'] = [father, son]
                json.dump(son_examples, out)
                out.write('\n')                

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1649/1649 [00:00<00:00, 35348.75it/s]
